### Set the current directory with the Location Yolo Files

In [1]:
cd F:\Project\Object Recog. with YOLO

F:\Project\Object Recog. with YOLO


In [2]:
model_path='F:\Project\Object Recog. with YOLO'

### Set the Threshold for Confidence filter Weak Detections

In [3]:
min_COnfidence=0.5
NMS_Thres=0.4

In [4]:
import numpy as np
import cv2

### We Create a function to Detect Features we want, We can extract more features as well which are avilable in coco.names in YOLO

In [5]:
def detectFeatures(frame, net, ln, personIndx=0, bicycleIndx=0, carIndx=0, motorbikeIndx=0, busIndx=0):
    #Getting Height, Width of the current frame
    (H,W)=frame.shape[:2]
    #We'll store out predicitons with it's details such as confidence, bounding box points in results
    results=[]
    #Creating a blob of current frame
    blob=cv2.dnn.blobFromImage(frame, 1/255.0,(416,416), swapRB=True, crop=False)
    #Giving the blob as input to our model to process this frame
    net.setInput(blob)
    #Getting the output of our Model which contains predictions
    layerOutputs=net.forward(ln)
    #We'll Stores the bounding box co-ordinates, confidences, ID which corresponds to object it is in these for our predictions
    boxes=[]
    confidences=[]
    classIDs=[]
    #Iterate though our outputs and each detection in each output
    for output in layerOutputs:
        for detection in output:
            #Extract Confidence for each detection and it's ID for item it corresponds to
            scores=detection[5:]
            classID=np.argmax(scores)
            confidence=scores[classID]
            #We just want Persons, Bicycles, Cars, Bikes and Buses to be detected
            if (classID==personIndx or classID==bicycleIndx or classID==carIndx or classID==motorbikeIndx or classID==busIndx) and confidence>min_COnfidence:
                #Get the boundary box co-ordinates for our Detectection
                box=detection[0:4]*np.array([W,H,W,H])
                (centerX, centerY, width, height)=box.astype('int')
                x=int(centerX-(width/2))
                y=int(centerY-(height/2))
                #Save these co-ordinates, confidence, ID of item number in the lists we initialized
                boxes.append([x,y,int(width),int(height)])
                confidences.append(float(confidence))
                classIDs.append(classID)
    #Applying Non-Max Suppression to remove extra Bounding Boxes
    indxs=cv2.dnn.NMSBoxes(boxes, confidences, min_COnfidence, NMS_Thres)
    #Storing Final Detection's features in results
    if len(indxs)>0:
        for i in indxs.flatten():
            (x,y)=(boxes[i][0], boxes[i][1])
            (w,h)=(boxes[i][2], boxes[i][3])
            r=(confidences[i], (x,y,x+w,y+h),classIDs[i])
            results.append(r)
    return results

In [6]:
import numpy as np
import imutils
import os

In [7]:
labelsPath='F:\Project\Object Recog. with YOLO\coco.names'
labels=open(labelsPath).read().strip().split('\n')
weightPaths='F:\Project\Object Recog. with YOLO\yolov3.weights'
configPath='F:\Project\Object Recog. with YOLO\yolov3.cfg'

### Creating our model

In [8]:
net=cv2.dnn.readNetFromDarknet(configPath, weightPaths)

In [9]:
cd F:\Project\Traffic Analyzer

F:\Project\Traffic Analyzer


### Giving the Input Video to process and output Location

In [10]:
inp='inp2.mp4'
output='output_01.avi'
display=0

In [11]:
ln=net.getLayerNames()
ln=[ln[i[0]-1] for i in net.getUnconnectedOutLayers()]
vs=cv2.VideoCapture(inp)
writer=None

In [ ]:
while True:
    #Reading frame from video
    (check, frame1) = vs.read()
    if not check:
        break
    frame=frame1.copy()
    #Resizing Frame
    frame = imutils.resize(frame, width=700)
    #Extracting Results from the above function
    results = detectFeatures(frame, net, ln,personIndx=labels.index('person'), bicycleIndx=labels.index('bicycle'), carIndx=labels.index('car'), motorbikeIndx=labels.index('motorbike'), busIndx=labels.index('bus'))
    #Setting Number of Vehicles initially 0
    twoWheeler=0
    fourWheeler=0
    person=0
    #Iterating through our results in Current frame
    for (i,(prob, bbox, classID))in enumerate(results):
        #Getting co-ordinates our current detection, and adding to corresponding Detection we got
        (startX, startY, endX, endY)=bbox
        if classID==labels.index('person'):
            person+=1
            color=(255,255,0)
        elif classID==labels.index('car') or classID==labels.index('bus'):
            fourWheeler+=1
            color=(0,255,0)
        elif classID==labels.index('bicycle') or classID==labels.index('motorbike'):
            twoWheeler+=1
            color=(100,100,100)
        #Plotting rectangle around the detection and displaying what it is
        cv2.rectangle(frame, (startX,startY),(endX,endY),color,1)
        cv2.putText(frame, labels[classID], (startX,startY-5),cv2.FONT_HERSHEY_SIMPLEX,0.5,color,1)
    #Displaying Summary of traffic on top-left
    text1='4-Wheelers: {}'.format(fourWheeler)
    text2='2-Wheelers: {}'.format(twoWheeler)
    text3='Pedestrians: {}'.format(person)
    cv2.putText(frame, text1, (25,frame.shape[0]-350),cv2.FONT_HERSHEY_SIMPLEX,0.6, (0,0,255),2)
    cv2.putText(frame, text2, (25,frame.shape[0]-325),cv2.FONT_HERSHEY_SIMPLEX,0.6, (0,0,255),2)
    cv2.putText(frame, text3, (25,frame.shape[0]-300),cv2.FONT_HERSHEY_SIMPLEX,0.6, (0,0,255),2)
    #If display>0, you'll get to see each frame while it's being processed and written to Output file
    if display>0:
        cv2.imshow('Frame',frame)
        key=cv2.waitKey(0)
        if key==27:
            break
        cv2.destroyAllWindows()
    if output!='' and writer is None:
        fourcc=cv2.VideoWriter_fourcc(*'MJPG')
        writer=cv2.VideoWriter(output, fourcc, 10, (frame.shape[1],frame.shape[0]))
    if writer is not None:
        writer.write(frame)
writer.release()